In [1]:
import os
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
import feather
import pandas as pd
import numpy as np
from scipy import stats
import gc
# from ipywidgets import IntProgress

In [2]:
def append_to_csv(batch, csv_file):
    props = dict(encoding='utf-8', index=False)
    if not os.path.exists(csv_file):
        batch.to_csv(csv_file, **props)
    else:
        batch.to_csv(csv_file, mode='a', header=False, **props)

def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [6]:
fn = '../cache/bkup/train10_t_store{}.csv'.format(1)
df =pd.read_csv(fn)

/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,item_family,item_class,perishable,city,...,dow,doy,dcount,h_type,h_desc,pd,wbe,wae,wfe,store_item_nbr
0,578,2013-01-02,1,103665,2.0,Unk,BREAD/BAKERY,2712,1,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_103665
1,579,2013-01-02,1,105574,8.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105574
2,580,2013-01-02,1,105575,15.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105575
3,581,2013-01-02,1,105577,2.0,Unk,GROCERY I,1045,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105577
4,582,2013-01-02,1,105737,2.0,Unk,GROCERY I,1044,0,Quito,...,2,2,734870,Work Day,Normal,0,172,0,172,1_105737


In [8]:
df.shape

(2562153, 27)

In [9]:
len(set(df.item_nbr))

3567

In [10]:
np.min(df.date)

'2013-01-02'

In [11]:
np.max(df.date)

'2017-08-15'

In [12]:
np.datetime64(np.max(df.date)) - np.datetime64(np.min(df.date))

numpy.timedelta64(1686,'D')

In [13]:
365 * 4

1460

In [14]:
def reset_recent_stats_promo2(df, days):
    df['us_{}_p_mean'.format(days)] = 0.
    df['us_{}_p_med'.format(days)] = 0.
    df['us_{}_p_std'.format(days)] = 0.
    df['us_{}_p_skew'.format(days)] = 0.
    df['us_{}_p_kurt'.format(days)]= 0.
    df['us_{}_p_tbn'.format(days)] = 0.
    return df

In [15]:
def set_recent_stats(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_mean'.format(days)] = mean
    df2.loc[0,'us_{}_med'.format(days)] = median
    df2.loc[0,'us_{}_std'.format(days)] = std
    df2.loc[0,'us_{}_skew'.format(days)] = skew
    df2.loc[0,'us_{}_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_promo(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_p_mean'.format(days)] = mean
    df2.loc[0,'us_{}_p_med'.format(days)] = median
    df2.loc[0,'us_{}_p_std'.format(days)] = std
    df2.loc[0,'us_{}_p_skew'.format(days)] = skew
    df2.loc[0,'us_{}_p_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_p_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_hol(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_h_mean'.format(days)] = mean
    df2.loc[0,'us_{}_h_med'.format(days)] = median
    df2.loc[0,'us_{}_h_std'.format(days)] = std
    df2.loc[0,'us_{}_h_skew'.format(days)] = skew
    df2.loc[0,'us_{}_h_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_h_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_per(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_per_mean'.format(days)] = mean
    df2.loc[0,'us_{}_per_med'.format(days)] = median
    df2.loc[0,'us_{}_per_std'.format(days)] = std
    df2.loc[0,'us_{}_per_skew'.format(days)] = skew
    df2.loc[0,'us_{}_per_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_per_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_nper(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0,'us_{}_nper_mean'.format(days)] = mean
    df2.loc[0,'us_{}_nper_med'.format(days)] = median
    df2.loc[0,'us_{}_nper_std'.format(days)] = std
    df2.loc[0,'us_{}_nper_skew'.format(days)] = skew
    df2.loc[0,'us_{}_nper_kurt'.format(days)] = kurtosis
    df2.loc[0,'us_{}_nper_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_dow(df2, date, days, dow, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_dow{}_mean'.format(days, dow)] = mean
    df2.loc[0, 'us_{}_dow{}_med'.format(days, dow)] = median
    df2.loc[0, 'us_{}_dow{}_std'.format(days, dow)] = std
    df2.loc[0, 'us_{}_dow{}_skew'.format(days, dow)] = skew
    df2.loc[0, 'us_{}_dow{}_kurt'.format(days, dow)] = kurtosis
    df2.loc[0, 'us_{}_dow{}_tbn'.format(days, dow)] = ten_by_ninety

def set_recent_stats_promo_hol(df2, date, days, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_ph_mean'.format(days)] = mean
    df2.loc[0, 'us_{}_ph_med'.format(days)] = median
    df2.loc[0, 'us_{}_ph_std'.format(days)] = std
    df2.loc[0, 'us_{}_ph_skew'.format(days)] = skew
    df2.loc[0, 'us_{}_ph_kurt'.format(days)] = kurtosis
    df2.loc[0, 'us_{}_ph_tbn'.format(days)] = ten_by_ninety


def set_recent_stats_dow_promo_hol(df1, date, days, dow, mean, median, std, skew, kurtosis, ten_by_ninety):
    df2.loc[0,'date'] = date
    df2.loc[0, 'us_{}_dow{}_ph_mean'.format(days, dow)] = mean
    df2.loc[0, 'us_{}_dow{}_ph_med'.format(days, dow)] = median
    df2.loc[0, 'us_{}_dow{}_ph_std'.format(days, dow)] = std
    df2.loc[0, 'us_{}_dow{}_ph_skew'.format(days, dow)] = skew
    df2.loc[0, 'us_{}_dow{}_ph_kurt'.format(days, dow)] = kurtosis
    df2.loc[0, 'us_{}_dow{}_ph_tbn'.format(days, dow)] = ten_by_ninety


In [16]:
# def set_recent_df_stats(df ,df1, days):
#     df['us_{}_mean'.format(days)] =  df1['us_{}_mean'.format(days)].values
#     df['us_{}_med'.format(days)] = df1['us_{}_med'.format(days)].values
#     df['us_{}_std'.format(days)] = df1['us_{}_std'.format(days)].values
#     df['us_{}_skew'.format(days)] = df1['us_{}_skew'.format(days)].values
#     df['us_{}_kurt'.format(days)] = df1['us_{}_kurt'.format(days)].values
#     df['us_{}_tbn'.format(days)] = df1['us_{}_tbn'.format(days)].values
#     return df

# def set_recent_df_stats_promo(df ,df1, days):
#     df['us_{}_p_mean'.format(days)] =  df1['us_{}_p_mean'.format(days)].values
#     df['us_{}_p_med'.format(days)] = df1['us_{}_p_med'.format(days)].values
#     df['us_{}_p_std'.format(days)] = df1['us_{}_p_std'.format(days)].values
#     df['us_{}_p_skew'.format(days)] = df1['us_{}_p_skew'.format(days)].values
#     df['us_{}_p_kurt'.format(days)] = df1['us_{}_p_kurt'.format(days)].values
#     df['us_{}_p_tbn'.format(days)] = df1['us_{}_p_tbn'.format(days)].values
#     return df

# def set_recent_df_stats_hol(df ,df1, days):
#     df['us_{}_h_mean'.format(days)] =  df1['us_{}_h_mean'.format(days)].values
#     df['us_{}_h_med'.format(days)] = df1['us_{}_h_med'.format(days)].values
#     df['us_{}_h_std'.format(days)] = df1['us_{}_h_std'.format(days)].values
#     df['us_{}_h_skew'.format(days)] = df1['us_{}_h_skew'.format(days)].values
#     df['us_{}_h_kurt'.format(days)] = df1['us_{}_h_kurt'.format(days)].values
#     df['us_{}_h_tbn'.format(days)] = df1['us_{}_h_tbn'.format(days)].values
#     return df

# def set_recent_df_stats_dow(df ,df1, days, dow):
#     df['us_{}_dow{}_mean'.format(days, dow)] =  df1['us_{}_dow{}_mean'.format(days, dow)].values
#     df['us_{}_dow{}_med'.format(days, dow)] = df1['us_{}_dow{}_med'.format(days, dow)].values
#     df['us_{}_dow{}_std'.format(days, dow)] = df1['us_{}_dow{}_std'.format(days, dow)].values
#     df['us_{}_dow{}_skew'.format(days, dow)] = df1['us_{}_dow{}_skew'.format(days, dow)].values
#     df['us_{}_dow{}_kurt'.format(days, dow)] = df1['us_{}_dow{}_kurt'.format(days, dow)].values
#     df['us_{}_dow{}_tbn'.format(days, dow)] = df1['us_{}_dow{}_tbn'.format(days, dow)].values
#     return df

# def set_recent_df_stats_promo_hol(df ,df1, days):
#     df['us_{}_ph_mean'.format(days)] =  df1['us_{}_ph_mean'.format(days)].values
#     df['us_{}_ph_med'.format(days)] = df1['us_{}_ph_med'.format(days)].values
#     df['us_{}_ph_std'.format(days)] = df1['us_{}_ph_std'.format(days)].values
#     df['us_{}_ph_skew'.format(days)] = df1['us_{}_ph_skew'.format(days)].values
#     df['us_{}_ph_kurt'.format(days)] = df1['us_{}_ph_kurt'.format(days)].values
#     df['us_{}_ph_tbn'.format(days)] = df1['us_{}_ph_tbn'.format(days)].values
#     return df

# def set_recent_df_stats_dow_promo_hol(df ,df1, days, dow):
#     df['us_{}_dow{}_ph_mean'.format(days, dow)] =  df1['us_{}_dow{}_ph_mean'.format(days, dow)].values
#     df['us_{}_dow{}_ph_med'.format(days, dow)] = df1['us_{}_dow{}_ph_med'.format(days, dow)].values
#     df['us_{}_dow{}_ph_std'.format(days, dow)] = df1['us_{}_dow{}_ph_std'.format(days, dow)].values
#     df['us_{}_dow{}_ph_skew'.format(days, dow)] = df1['us_{}_dow{}_ph_skew'.format(days, dow)].values
#     df['us_{}_dow{}_ph_kurt'.format(days, dow)] = df1['us_{}_dow{}_ph_kurt'.format(days, dow)].values
#     df['us_{}_dow{}_ph_tbn'.format(days, dow)] = df1['us_{}_dow{}_ph_tbn'.format(days, dow)].values
#     return df

In [ ]:
# recent_days = [730, 365, 180, 90, 30]

In [18]:
recent_days = [30, 90, 180, 365, 730]

In [19]:
def calculate_recent_stats(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        

def calculate_recent_stats_promo(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_promo(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
#         data = df.unit_sales[(df.dts < np.datetime64(date)) & 
#                              (df.dts >= past_date) & (df.onpromotion == True)]
        data = df.unit_sales[(df.date.astype('str') < str(date)) & 
                             (df.date.astype('str') >= str(past_date)) & (df.onpromotion == True) ]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_promo(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_promo(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
        
def calculate_recent_stats_hol(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.h_type != 'Work Day')]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_hol(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
 
def calculate_recent_stats_perishable(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_per(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.perishable == 1)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_per(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_per(df3, date,days, mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
def calculate_recent_stats_not_perishable(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_nper(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.perishable == 0)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_nper(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_nper(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
        
def calculate_recent_stats_dow(date, dow, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & (df.dow == dow)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_dow(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
def calculate_recent_stats_promo_hol(date, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & 
                             (df.h_type != 'Work Day') & (df.onpromotion == True)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats(df3, date,days,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_promo_hol(df3, date,days,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3
    
def calculate_recent_stats_dow_promo_hol(date, dow, df, fn):
    df3 = pd.DataFrame()
    for days in recent_days:
        past_date = (np.datetime64(date) - np.timedelta64(days, 'D'))
        i = 0
        df_min_date = np.datetime64(np.min(df.date))
        while (past_date < df_min_date) & (i <= 0):
            past_date = (np.datetime64(date) - np.timedelta64(days-i, 'D'))
            i +=1
        
        if past_date < df_min_date:
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_dow_promo_hol(df3, date,days,dow,0, 0, 0, 0, 0, 0)
            continue
        
        data = df.unit_sales[(df.dts < np.datetime64(date)) & 
                             (df.dts >= past_date) & 
                             (df.h_type != 'Work Day') & 
                             (df.onpromotion == True) & 
                             (df.dow == dow)]
        if(data.shape[0] == 0):
            print('date:{}, past_date:{}'.format(date, past_date))
            j = days-i
            print('days:{}, i: {}, days-i:{}'.format(days, i, j))
            set_recent_stats_dow_promo_hol(df3, date,days,dow,0, 0, 0, 0, 0, 0)
            continue
        mean = data.mean()
        median = data.median()
        std = data.std()
        skew = data.skew()
        kurtosis = data.kurtosis()
        tenp = data.quantile(0.1)
        ninetyp = data.quantile(0.9)
        ten_by_ninety = tenp/ninetyp
        set_recent_stats_dow_promo_hol(df3, date,days,dow,mean, median, std, skew, kurtosis, ten_by_ninety)
        del data
    append_to_csv(df3, fn)
    del df3

In [20]:
print(df.unit_sales.mean())

5.52075222249


In [21]:
past_date = (np.datetime64('2016-01-01') - np.timedelta64(2, 'D'))

In [22]:
# df.unit_sales[(df.date.apply(np.datetime64) >= past_date) & 
#               (df.date.apply(np.datetime64) < np.datetime64('2016-01-01'))].mean()

ValueError: cannot set a Timestamp with a non-timestamp

In [24]:
df1 = df.set_index('date')

In [25]:
s1 = df1.loc['2015-12-31', 'unit_sales']
s2 = df1.loc['2015-12-30', 'unit_sales']

In [26]:
s = pd.concat([s1, s2], axis=0)

In [27]:
s.mean()

6.5914254756871031

In [28]:
gc.enable()

In [29]:
# from multiprocessing import Pool, cpu_count
def process_csv_file2(i):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn)
    df = pd.read_csv(fn)
    print(df.shape)
    df.unit_sales[df.unit_sales < 0] = 0 
    df = df.iloc[::-1]
    df1 = df.set_index('date')

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    for date in tqdm(trend_dates):
        calculate_recent_stats_promo(date, df, df1)
    for days in [30, 90, 180, 365, 730]:
        df = set_recent_df_stats_promo(df, df1, days)
    df = df.iloc[::-1]
    
    df = df[(df.date.apply(np.datetime64) >= np.datetime64('2015-01-01')) & 
              (df.date.apply(np.datetime64) <= np.datetime64('2017-08-15'))]
    
    print(df.shape)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats_promo.csv'.format(i)
    print(fn)
    df.to_csv(fn, index=False)
    del df
    del df1

In [30]:
def process_csv_file1(i):
    fn = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn)
    df = pd.read_csv(fn)

    df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    fn = '../cache/train10_t_store{}_recent_stats_promo_tmp.csv'.format(i)
    for date in tqdm(trend_dates):
        calculate_recent_stats_promo2(date, df, fn)

    
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]

    print(df.shape)
    
    df1 = pd.read_csv(fn)
    print(df1.shape)
    df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats_promo_my.csv'.format(i)
    print(fn)
    df.to_csv(fn, index=False)
    del df
    del df1

In [31]:
def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [32]:
def process_csv_file1(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn_org)
    df = pd.read_csv(fn_org)
    
    df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    fn_tmp = '../cache/train10_t_store{}_recent_stats_promo_tmp.csv'.format(i)
    delete_file_if_exists(fn_tmp)
    for date in tqdm(trend_dates):
        calculate_recent_stats_promo(date, df, fn_tmp)

    print(df.shape)
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]

    print(df.shape)
    nrows1 = df.shape[0]
    df1 = pd.read_csv(fn_tmp)
    print(df1.shape)
    df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats_promo.csv'.format(i)
    print(fn)
    df.to_csv(fn, index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1

In [33]:
def process_csv_file(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    print(fn_org)
    df = pd.read_csv(fn_org)
    
#     df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2015-01-01')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    fn_tmp = '../cache/train10_t_store{}_recent_stats_promo_tmp.csv'.format(i)
    delete_file_if_exists(fn_tmp)
    for date in tqdm(trend_dates):
        calculate_recent_stats_promo(date, df, fn_tmp)

    print(df.shape)
    df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]

    print(df.shape)
    nrows1 = df.shape[0]
    df1 = pd.read_csv(fn_tmp)
    print(df1.shape)
    df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/train10_t_store{}_recent_stats_promo_1.csv'.format(i)
    print(fn)
    df.to_csv(fn, float_format='%.5f',index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1
    delete_file_if_exists(fn_tmp)

In [34]:
def process_test_csv_file(i):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(i)
    fn_org1 = '../cache/bkup2/test7_1_t_store{}.csv'.format(i)
    print(fn_org1)
    df = pd.read_csv(fn_org)
    df1 = pd.read_csv(fn_org1)
    df = pd.concat([df, df1], axis=0)
    
#     df.unit_sales[df.unit_sales < 0] = 0 
    df['dts'] = df.date.map(np.datetime64)

    trend_dates = []
    for date in np.unique(df.date.values):
        curr = np.datetime64(date)
        last_in_trend = np.datetime64('2017-08-16')
        if curr < last_in_trend:
            continue
        trend_dates.append(date)
    
    fn_tmp = '../cache/test7_t_store{}_recent_stats_promo_tmp.csv'.format(i)
    delete_file_if_exists(fn_tmp)
    for date in tqdm(trend_dates):
        calculate_recent_stats_promo(date, df, fn_tmp)

    print(df.shape)
#     df = df[(df.dts >= np.datetime64('2015-01-01')) & (df.dts <= np.datetime64('2017-08-15'))]
    df = df[(df.date.astype('str') >= str('2017-08-16')) & (df.date.astype('str') <= str('2017-08-31'))]

    print(df.shape)
    nrows1 = df.shape[0]
    df1 = pd.read_csv(fn_tmp)
    print(df1.shape)
    df = pd.merge(df, df1, how='left', on='date')
    df = df.drop('dts', axis=1)
    s = sum(df.isnull().any()==True) # print NA
    print(s)
    if s > 0:
        print(df.isnull().sum())
    print(sum((df != 0).any(axis=0))) # print non-zero columns
    fn = '../cache/test7_1_t_store{}_recent_stats_promo_1.csv'.format(i)
    print(fn)
    df.to_csv(fn, float_format='%.5f',index=False)
    print(df.shape)
    nrows2 = df.shape[0]
    assert(nrows1 == nrows2)
    del df
    del df1
    delete_file_if_exists(fn_tmp)

In [35]:
# for i in range(1,55):
#     process_test_csv_file(i)

../cache/bkup2/test7_1_t_store1.csv


/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
100%|██████████| 16/16 [01:50<00:00,  6.89s/it]


(2624569, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store1_recent_stats_promo_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store2.csv


100%|██████████| 16/16 [02:08<00:00,  8.00s/it]


(3050256, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store2_recent_stats_promo_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store3.csv


...

(62416, 28)
(16, 31)
0
38
../cache/test7_1_t_store52_recent_stats_promo_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store53.csv


100%|██████████| 16/16 [01:14<00:00,  4.64s/it]


(2000671, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store53_recent_stats_promo_1.csv
(62416, 57)
../cache/bkup2/test7_1_t_store54.csv


100%|██████████| 16/16 [01:12<00:00,  4.51s/it]


(1711283, 28)
(62416, 28)
(16, 31)
0
56
../cache/test7_1_t_store54_recent_stats_promo_1.csv
(62416, 57)


In [4]:
date_block = {}
date_block['1_2013'] = 0
date_block['2_2013'] = 1
date_block['3_2013'] = 2
date_block['4_2013'] = 3
date_block['5_2013'] = 4
date_block['6_2013'] = 5
date_block['7_2013'] = 6
date_block['8_2013'] = 7
date_block['9_2013'] = 8
date_block['10_2013'] = 9
date_block['11_2013'] = 10
date_block['12_2013'] = 11

date_block['1_2014'] = 12
date_block['2_2014'] = 13
date_block['3_2014'] = 14
date_block['4_2014'] = 15
date_block['5_2014'] = 16
date_block['6_2014'] = 17
date_block['7_2014'] = 18
date_block['8_2014'] = 19
date_block['9_2014'] = 20
date_block['10_2014'] = 21
date_block['11_2014'] = 22
date_block['12_2014'] = 23

date_block['1_2015'] = 24
date_block['2_2015'] = 25
date_block['3_2015'] = 26
date_block['4_2015'] = 27
date_block['5_2015'] = 28
date_block['6_2015'] = 29
date_block['7_2015'] = 30
date_block['8_2015'] = 31
date_block['9_2015'] = 32
date_block['10_2015'] = 33
date_block['11_2015'] = 34
date_block['12_2015'] = 35

date_block['1_2016'] = 36
date_block['2_2016'] = 37
date_block['3_2016'] = 38
date_block['4_2016'] = 39
date_block['5_2016'] = 40
date_block['6_2016'] = 41
date_block['7_2016'] = 42
date_block['8_2016'] = 43
date_block['9_2016'] = 44
date_block['10_2016'] = 45
date_block['11_2016'] = 46
date_block['12_2016'] = 47

date_block['1_2017'] = 48
date_block['2_2017'] = 49
date_block['3_2017'] = 50
date_block['4_2017'] = 51
date_block['5_2017'] = 52
date_block['6_2017'] = 53
date_block['7_2017'] = 54
date_block['8_2017'] = 55
date_block['9_2017'] = 56
date_block['10_2017'] = 57
date_block['11_2017'] = 58
date_block['12_2017'] = 59

def get_date_block_num(x):
    year, month, day = x.split('-')
    mon = int(month)
    yr = int(year)
    mon_year = str(mon) + '_' + str(yr)
    
    return date_block[mon_year]


In [15]:
for shop_num in tqdm(range(1,55)):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(shop_num)
    fn_org1 = '../cache/bkup2/test7_1_t_store{}.csv'.format(shop_num)
    print(fn_org1)
    df = pd.read_csv(fn_org)
    df1 = pd.read_csv(fn_org1)
    df = pd.concat([df, df1], axis=0)
    
#     print(df.shape)
    df['date_block_num']  = df.date.map(lambda x: get_date_block_num(x))
    
    df['in_op']  = df.item_nbr.astype('str') + '_' + df.onpromotion.astype('str')
#     df = df.drop_duplicates()
    unit_sales_promo_mean = df.groupby(['in_op', 'date_block_num'], as_index=False).agg({'unit_sales':{'unit_sales_promo_mean':'mean'}, 
                                                         })
    unit_sales_promo_mean.columns = ['in_op', 'date_block_num', 'unit_sales_promo_mean']
    df = pd.merge(df, unit_sales_promo_mean, how='left', on=['in_op', 'date_block_num']).fillna(0.0)

#     unit_sales_promo_median = df.groupby(['in_op', 'date_block_num'], as_index=False).agg({'unit_sales':{'unit_sales_promo_median':'median'}, 
#                                                          })
#     unit_sales_promo_median.columns = ['in_op', 'date_block_num', 'unit_sales_promo_median']
#     df = pd.merge(df, unit_sales_promo_median, how='left', on=['item_nbr', 'onpromotion', 'date_block_num']).fillna(0.0)
    
#     unit_sales_promo_std = df.groupby(['in_op', 'date_block_num'], as_index=False).agg({'unit_sales':{'unit_sales_promo_std':'std'}, 
#                                                          })
#     unit_sales_promo_std.columns = ['in_op', 'date_block_num', 'unit_sales_promo_std']
#     df = pd.merge(df, unit_sales_promo_std, how='left', on=['item_nbr', 'onpromotion', 'date_block_num']).fillna(0.0)
    
    
#     shifted_df = df[['id', 'date', 'date_block_num','item_nbr','onpromotion','unit_sales_promo_mean', 'unit_sales_promo_median', 'unit_sales_promo_std']].copy()
    
    shifted_df = df[['id', 'date', 'date_block_num','in_op','unit_sales_promo_mean']].copy()
    
    
    fn = '../cache/shifted_rsp_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    
    shifted_df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df
    del unit_sales_promo_mean
#     del unit_sales_promo_median
#     del unit_sales_promo_std


  0%|          | 0/54 [00:00<?, ?it/s]

../cache/bkup2/test7_1_t_store1.csv


/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/pandas/core/groupby.py:4189: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)
  2%|▏         | 1/54 [00:29<26:02, 29.48s/it]

../cache/bkup2/test7_1_t_store2.csv


  4%|▎         | 2/54 [01:03<27:23, 31.60s/it]

...

 96%|█████████▋| 52/54 [23:21<00:53, 26.96s/it]

../cache/bkup2/test7_1_t_store53.csv


 98%|█████████▊| 53/54 [23:47<00:26, 26.93s/it]

../cache/bkup2/test7_1_t_store54.csv


100%|██████████| 54/54 [24:06<00:00, 26.78s/it]


In [16]:


# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rsp_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num', 'in_op','unit_sales_promo_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 1
    shifted_df.columns = ['date_block_num', 'in_op', 'unit_sales_promo_mean_f_lag_1']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num', 'in_op'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    print(len(df1), len(shifted_df))
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_promo_mean_f_lag_1'] = df1.unit_sales_promo_mean_f_lag_1.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag1_sales_mean_rsp_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df


  0%|          | 0/54 [00:00<?, ?it/s]

2624569 2624569


  2%|▏         | 1/54 [00:25<22:18, 25.25s/it]

3050256 3050256


  4%|▎         | 2/54 [00:56<24:30, 28.27s/it]

...

 96%|█████████▋| 52/54 [20:55<00:48, 24.15s/it]

2000671 2000671


 98%|█████████▊| 53/54 [21:14<00:24, 24.04s/it]

1711283 1711283


100%|██████████| 54/54 [21:29<00:00, 23.89s/it]


In [17]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rsp_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num', 'in_op','unit_sales_promo_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 3
    shifted_df.columns = ['date_block_num', 'in_op', 'unit_sales_promo_mean_f_lag_3']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num', 'in_op'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    print(len(df1), len(shifted_df))
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_promo_mean_f_lag_3'] = df1.unit_sales_promo_mean_f_lag_3.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag3_sales_mean_rsp_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df


  0%|          | 0/54 [00:00<?, ?it/s]

2624569 2624569


  2%|▏         | 1/54 [00:23<20:56, 23.72s/it]

3050256 3050256


  4%|▎         | 2/54 [00:52<22:57, 26.48s/it]

...

 96%|█████████▋| 52/54 [20:44<00:47, 23.93s/it]

2000671 2000671


 98%|█████████▊| 53/54 [21:21<00:24, 24.18s/it]

1711283 1711283


100%|██████████| 54/54 [22:22<00:00, 24.86s/it]


In [18]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rsp_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num', 'in_op','unit_sales_promo_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 6
    shifted_df.columns = ['date_block_num', 'in_op', 'unit_sales_promo_mean_f_lag_6']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num', 'in_op'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    print(len(df1), len(shifted_df))
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_promo_mean_f_lag_6'] = df1.unit_sales_promo_mean_f_lag_6.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag6_sales_mean_rsp_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df


  0%|          | 0/54 [00:00<?, ?it/s]

2624569 2624569


  2%|▏         | 1/54 [00:58<51:35, 58.40s/it]

3050256 3050256


  4%|▎         | 2/54 [02:50<1:13:55, 85.30s/it]

...

 96%|█████████▋| 52/54 [33:28<01:17, 38.63s/it]

2000671 2000671


 98%|█████████▊| 53/54 [33:45<00:38, 38.22s/it]

1711283 1711283


100%|██████████| 54/54 [33:59<00:00, 37.77s/it]


In [19]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rsp_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num', 'in_op','unit_sales_promo_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 12
    shifted_df.columns = ['date_block_num', 'in_op', 'unit_sales_promo_mean_f_lag_12']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num', 'in_op'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    print(len(df1), len(shifted_df))
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_promo_mean_f_lag_12'] = df1.unit_sales_promo_mean_f_lag_12.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag12_sales_mean_rsp_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df


  0%|          | 0/54 [00:00<?, ?it/s]

2624569 2624569


  2%|▏         | 1/54 [00:19<17:14, 19.51s/it]

3050256 3050256


  4%|▎         | 2/54 [00:43<18:46, 21.66s/it]

...

 96%|█████████▋| 52/54 [15:14<00:35, 17.59s/it]

2000671 2000671


 98%|█████████▊| 53/54 [15:28<00:17, 17.53s/it]

1711283 1711283


100%|██████████| 54/54 [15:40<00:00, 17.41s/it]


In [20]:
# from tqdm import tqdm
for shop_num in tqdm(range(1,55)):
    fn = '../cache/shifted_rsp_df_shop{}.csv'.format(shop_num)
    df1 = pd.read_csv(fn)

#     print(df1.shape)
#     print(set(df1.date_block_num))
#     df1 = df1.drop('my_id', axis=1)
    # forward
    # 1 month
    shifted_df = df1[['date_block_num', 'in_op','unit_sales_promo_mean']].copy()
#     print(shifted_df.shape)
    org_date_blocks = shifted_df.date_block_num.values
    shifted_df.date_block_num += 24
    shifted_df.columns = ['date_block_num', 'in_op', 'unit_sales_promo_mean_f_lag_24']
#     print(shifted_df.date_block_num.nunique())
    
    df1 = df1.merge(shifted_df, on=['date_block_num', 'in_op'], how='left').fillna(0.0)
    df1 = df1.drop_duplicates()
    print(len(df1), len(shifted_df))
    assert(len(df1) == len(shifted_df))
    
    df = pd.DataFrame()
    df['id'] = df1.id.values
    df['unit_sales_promo_mean_f_lag_24'] = df1.unit_sales_promo_mean_f_lag_24.values
    df = df.loc[:, (df != 0).any(axis=0)]
    if len(df.columns) != 2:
        print(shop_num)
        print(df1.shape)
        print(set(df1.date_block_num))
        print(shifted_df.date_block_num.nunique())
        print(set(shifted_df.date_block_num))
        print(df.shape)
        print('------')
    fn = '../cache/lag24_sales_mean_rsp_df_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    df.to_csv(fn, float_format='%.5f', index=False)
    del shifted_df
    del df1
    del df


  0%|          | 0/54 [00:00<?, ?it/s]

2624569 2624569


  2%|▏         | 1/54 [00:13<12:16, 13.90s/it]

3050256 3050256


  4%|▎         | 2/54 [00:30<13:19, 15.38s/it]

...

 96%|█████████▋| 52/54 [11:04<00:25, 12.79s/it]

2000671 2000671


 98%|█████████▊| 53/54 [11:15<00:12, 12.75s/it]

1711283 1711283


100%|██████████| 54/54 [11:24<00:00, 12.68s/it]


In [23]:
for shop_num in tqdm(range(1,55)):
    df1 = pd.DataFrame()
    
    df1['unit_sales_promo_mean_f_lag_1'] =0
    df1['unit_sales_promo_mean_f_lag_3'] = 0
    df1['unit_sales_promo_mean_f_lag_6'] = 0
    df1['unit_sales_promo_mean_f_lag_12'] = 0
    df1['unit_sales_promo_mean_f_lag_24'] = 0
    
    fn = '../cache/lag1_sales_mean_rsp_df_shop{}.csv'.format(shop_num)
    df2 = pd.read_csv(fn)
    
    fn = '../cache/lag3_sales_mean_rsp_df_shop{}.csv'.format(shop_num)
    df3 = pd.read_csv(fn)
    
    fn = '../cache/lag6_sales_mean_rsp_df_shop{}.csv'.format(shop_num)
    df4 = pd.read_csv(fn)
    
    fn = '../cache/lag12_sales_mean_rsp_df_shop{}.csv'.format(shop_num)
    df5 = pd.read_csv(fn)
    
    fn = '../cache/lag24_sales_mean_rsp_df_shop{}.csv'.format(shop_num)
    df6 = pd.read_csv(fn)
    
    df1['id'] = df2['id'].values
    if len(df2.columns) > 1:
        df1['unit_sales_promo_mean_f_lag_1'] = df2.unit_sales_promo_mean_f_lag_1.values
        
    if len(df3.columns) > 1:
        df1['unit_sales_promo_mean_f_lag_3'] = df3.unit_sales_promo_mean_f_lag_3.values
        
        
    if len(df4.columns) > 1:
        df1['unit_sales_promo_mean_f_lag_6'] = df4.unit_sales_promo_mean_f_lag_6.values
        
    if len(df5.columns) > 1:
        df1['unit_sales_promo_mean_f_lag_12'] = df5.unit_sales_promo_mean_f_lag_12.values
        
    if len(df6.columns) > 1:
        df1['unit_sales_promo_mean_f_lag_24'] = df6.unit_sales_promo_mean_f_lag_24.values
     
    del df2, df3, df4, df5, df6
    
    fn = '../cache/df_all_rsp_lag_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn)
    
#     const_col = []
#     for col in df1.columns:
#         c = len(set(df1[col].values))
#         if c==1:
#             const_col.append(col)
#     if len(const_col) > 0:
#         c = len(const_col)
#         print(shop_num)
#         print(c)
#         print(df1.shape)
#         print(const_col)
    df1.to_csv(fn, float_format='%.5f', index=False)
    del df1
    print('5..')
    gc.collect()

  0%|          | 0/54 [00:00<?, ?it/s]

5..


  4%|▎         | 2/54 [00:53<23:10, 26.75s/it]

5..


  6%|▌         | 3/54 [01:25<24:18, 28.60s/it]

...

5..


 98%|█████████▊| 53/54 [22:59<00:26, 26.03s/it]

5..


100%|██████████| 54/54 [23:16<00:00, 25.86s/it]

5..


In [3]:
for shop_num in tqdm(range(1,55)):
    fn_org = '../cache/bkup/train10_t_store{}.csv'.format(shop_num)
    fn_org1 = '../cache/bkup2/test7_1_t_store{}.csv'.format(shop_num)
    print(fn_org1)
    train_org = pd.read_csv(fn_org)
    test_org = pd.read_csv(fn_org1)
    df_shop_all = pd.concat([train_org, test_org], axis=0)
    
#     fn_org = '../cache/bkup/train10_t_store{}.csv'.format(shop_num)
#     train_org = pd.read_csv(fn_org)
    
    fn = '../cache/df_all_rsp_lag_shop{}.csv'.format(shop_num)
    df = pd.read_csv(fn)
    df['date'] = df_shop_all.date.values
    
    df_train = df[:len(train_org)]
    df_train = df_train[(df_train.date.astype('str') >= str('2015-01-01')) & 
                        (df_train.date.astype('str') <= str('2017-08-15'))]
    df_train = df_train.drop('date', axis=1)
    
    df_test = df[len(train_org):]
    df_test = df_test.drop('date', axis=1)
    assert(len(test_org) == len(df_test))
    
    fn_tr = '../cache/df_train10_t_rsp_lag_shop{}.csv'.format(shop_num)
    fn_te = '../cache/df_test7_1_rsp_lag_shop{}.csv'.format(shop_num)
    delete_file_if_exists(fn_tr)
    delete_file_if_exists(fn_te)
    
    df_train.to_csv(fn_tr, index=False, float_format='%.5f')
    df_test.to_csv(fn_te, index=False, float_format='%.5f')
    
    del df_shop_all
    del train_org, test_org
    del df_train
    del df_test

  0%|          | 0/54 [00:00<?, ?it/s]

../cache/bkup2/test7_1_t_store1.csv


/home/watts/anaconda2/envs/sdp35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
  2%|▏         | 1/54 [00:44<39:42, 44.95s/it]

../cache/bkup2/test7_1_t_store2.csv


  4%|▎         | 2/54 [01:42<44:17, 51.10s/it]

...

 96%|█████████▋| 52/54 [36:46<01:24, 42.43s/it]

../cache/bkup2/test7_1_t_store53.csv


 98%|█████████▊| 53/54 [37:27<00:42, 42.40s/it]

../cache/bkup2/test7_1_t_store54.csv


100%|██████████| 54/54 [37:57<00:00, 42.18s/it]


In [ ]:
# i = 4
# fn = '../cache/train10_t_store{}_recent_stats_promo.csv'.format(i)
# print(fn)
# df = pd.read_csv(fn)
# fn = '../cache/train10_t_store{}_recent_stats_promo_my.csv'.format(i)
# df1 = pd.read_csv(fn)
# df1 = df1[df.columns]

In [ ]:
# from pandas.util.testing import assert_frame_equal
# assert_frame_equal(df, df1)

In [ ]:
# df.isnull().sum()

In [ ]:
# for i in range(1,55):
#     print(i)
#     fn = '../cache/train10_t_store{}.csv'.format(i)
#     df = pd.read_csv(fn)
#     df.unit_sales[df.unit_sales < 0] = 0
#     df = df.iloc[::-1]
#     df1 = df.set_index('date')
#     dates = np.unique(df.date.values)
#     for j, date in tqdm(enumerate(dates)):
#         calculate_recent_stats_promo(date, df, df1)
#     for days in [30, 90, 180, 365, 730]:
#         df = set_recent_df_stats_promo(df, df1, days)
#     df = df.iloc[::-1]
#     print(df.shape)
#     print(sum(df.isnull().any()==True))
#     print(sum((df != 0).any(axis=0)))
#     fn = '../cache/train10_t_store{}_recent_stats_ptomo.csv'.format(i)
#     print(fn)
#     df.to_csv(fn, index=False)
#     del df
#     del df1

In [ ]:
gc.collect()

In [ ]:
# i = 1
# fn = '../cache/train10_t_store{}.csv'.format(i)
# df = pd.read_csv(fn)
# df.unit_sales[df.unit_sales < 0] = 0
# df1 = df.set_index('date')
# dates = np.unique(df.date.values)
# for j, date in tqdm(enumerate(dates)):
#     if j > 0:
#         break
#     print(j)
#     calculate_recent_stats(date, df, df1)
# for days in [15, 30, 60, 90, 180, 365, 730, 1095, 1460]:
#     df = set_recent_df_stats(df ,df1, days)
# print(df.shape)
# print(sum(df.isnull().any()==True))
# print(sum((df != 0).any(axis=0)))
# fn = '../cache/train10_t_store{}_recent_stats.csv'.format(i)
# print(fn)

In [ ]:
# df.isnull().sum()

In [ ]:
# for i in tqdm(range(1,55)):
#     fn_tmp = '../cache/train10_t_store{}_recent_stats_promo_tmp.csv'.format(i)
#     delete_file_if_exists(fn_tmp)
#     fn = '../cache/train10_t_store{}_recent_stats_promo.csv'.format(i)
#     df = pd.read_csv(fn)
#     df.to_csv(fn, float_format='%.5f', index=False)
#     del df

In [3]:
from scipy.sparse import save_npz, load_npz
import sys

sys.stdout = open('/dev/stdout', 'w')
new_cols = []

In [4]:
recent_days = [30, 90, 180, 365, 730]
for days in tqdm(recent_days):
    new_cols.append('us_{}_p_mean'.format(days))
    new_cols.append('us_{}_p_med'.format(days))
    new_cols.append('us_{}_p_std'.format(days))
    new_cols.append('us_{}_p_skew'.format(days))
    new_cols.append('us_{}_p_kurt'.format(days))
    new_cols.append('us_{}_p_tbn'.format(days))
    

100%|██████████| 5/5 [00:00<00:00, 17232.14it/s]


In [5]:
df_test_rsp = pd.DataFrame()
for store_num in tqdm(range(1,55)):
    fn = '../cache/test7_1_t_store{}_recent_stats_promo_1.csv'.format(store_num)
    df = pd.read_csv(fn)
    df_test_rsp = pd.concat([df_test_rsp, df], axis=0)
    del df


100%|██████████| 54/54 [01:06<00:00,  1.24s/it]


In [6]:
df_test_rsp = df_test_rsp.sort_values('id')
df_test_rsp = df_test_rsp[new_cols]

In [7]:
df_test_rsp.shape

(3370464, 30)

In [8]:
delete_file_if_exists('../cache/df_test_rsp.csv')
df_test_rsp.to_csv('../cache/df_test_rsp.csv', index=False, float_format='%.5f')

In [9]:
X_test_rsp = df_test_rsp.values
np.save('../cache/X_test_rsp', X_test_rsp)